# Chapter 5. Estimating Counts

[Link to chapter online](https://allendowney.github.io/ThinkBayes2/chap05.html)


In this chapter, we’ll work on problems related to counting, or estimating the size of a population.

A reminder of Bayes’s Theorem:

$P(A|B) = \frac{P(A)P(B|A)}{P(B)}$

or

$P(H|D) = \frac{P(H)P(D|H)}{P(D)}$